In [2]:
from pdp_utils import *
import random
import numpy as np
import functools

np.random.seed(20)

def resolve_max_capacity(prob, vehicle_idx):
    capacities = prob["VesselCapacity"]
    try:
        return capacities[vehicle_idx]
    # Index out of bounds: give dummy truck infinite capacity
    except:
        return float('inf')
    
def resolve_allowed_calls(prob, vehicle_idx):
    allowed_calls_list = prob["VesselCargo"]
    try:
        return allowed_calls_list[vehicle_idx]
    # Index out of bounds, allow dummy truck all calls
    except:
        return np.array([1. for i in range(prob["n_calls"])])

    


In [7]:
class Truck:
    
    def __init__(self, idx, max_capacity, allowed_calls):
        self.idx = idx
        self.max_capacity = max_capacity
        self.allowed_calls = allowed_calls
        self.actions = []
        self.active = True
        
    def take_call(self, call_idx, prob):
        self.actions.append(call_idx)
        pass
    
    def resolve_complete_call_time(self, call_idx, prob):
        return 0
    
    def resolve_complete_call_cost(self, call_idx, prob):
        return 0
    
        

In [8]:
def generate_trucks(prob):
    trucks = []
    for i in range(prob["n_vehicles"]):
        initial_call = np.random.choice(np.size(prob["Cargo"], 0))
        max_capacity = resolve_max_capacity(prob, i)
        allowed_calls = resolve_allowed_calls(prob, i)    
        truck = Truck(i, max_capacity, allowed_calls)
        trucks.append(truck)
    dummy_truck = Truck(prob["n_vehicles"], float('inf'), np.array([1 for i in range(len(prob["VesselCargo"][0]))]))
    return trucks, dummy_truck

In [29]:
def run_problem(problem):
    prob_take_call = 0.5
    prob_retire_truck = 0.1
    prob = load_problem(problem)
    cargos = prob["Cargo"].copy()
    trucks, dummy_truck = generate_trucks(prob)
    available_trucks = trucks.copy()
    rejected_calls_cost = 0
    
    # Distribute calls between trucks
    while len(cargos) > 0:
        to_select = np.random.choice(np.size(cargos, 0))
        current_call = cargos[to_select]
        call_idx = np.where(np.all(prob["Cargo"] == current_call,axis=1))[0][0]
        
        # Assign selected call to random truck, with prob_take_call probability of rejecting it.
        available_trucks = [truck for truck in available_trucks if truck.active]
        take_call_value = random.random()
        if(len(available_trucks) > 0 and take_call_value < prob_take_call):
            # Some truck can make the call: Select random truck among available trucks.
            truck = random.choice(available_trucks)
            truck.take_call(call_idx, prob)
        else:
            # Do not take call - give call to dummy truck, increment cost of rejected calls
            dummy_truck.take_call(call_idx, prob)
            rejected_calls_cost += prob["Cargo"][call_idx][3]

        # Remove call from list of calls
        cargos = np.delete(cargos, to_select, axis=0)
        
        # Give trucks chance to accept no more calls with probability prob_retire_truck
        for truck in available_trucks:
            retire_truck_value = random.random()
            if(retire_truck_value < prob_retire_truck):
                truck.active = False
    
    # Let trucks decide in which order calls should be resolved
    for truck in (trucks):
        calls = truck.actions.copy()
        for call in calls:
            c_idx = truck.actions.index(call) + 1
            insert_idx = random.randint(c_idx, len(truck.actions))
            truck.actions.insert(insert_idx, call)

        
    solution = functools.reduce(lambda acc, val: acc + [i + 1 for i in val.actions] + [0], trucks, [])
    solution = solution + [i+1 for i in dummy_truck.actions]
    feasiblity, c = feasibility_check(solution, prob)
    sol_cost = cost_function(solution, prob) + rejected_calls_cost
    return solution, sol_cost, feasiblity, c, prob
    
    

## C7V3

In [38]:
import time
n = 1000

start_time = time.time()

c7v3_init_sol, c7v3_init_cost, feasibility, c, init_prob = [], float('inf'), False, '', None
c7v3_best_sol = c7v3_init_sol
c7v3_best_cost = c7v3_init_cost
c7v3_total_cost = c7v3_init_cost
# Force start with feasible solution
while(not feasibility):
    c7v3_init_sol, c7v3_init_cost, feasibility, c, init_prob = run_problem('./Call_7_Vehicle_3.txt')
    c7v3_best_sol = c7v3_init_sol
    c7v3_best_cost = c7v3_init_cost
    c7v3_total_cost = c7v3_init_cost

for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_7_Vehicle_3.txt')
    if(feasibility):
        c7v3_total_cost += sol_cost
        if(sol_cost < c7v3_best_cost):
            c7v3_best_sol = solution
            c7v3_best_cost = sol_cost
end_time = time.time()

c7v3_avg_cost = c7v3_total_cost / (n+1)
print(c7v3_best_sol)
print('init_cost', c7v3_init_cost)
print('best_cost', c7v3_best_cost)
print('avg_cost', c7v3_avg_cost)
print('improvement', ((c7v3_init_cost - c7v3_best_cost) / c7v3_init_cost)*100)
print('elapsed time', end_time - start_time)


[7, 7, 0, 3, 3, 0, 1, 5, 5, 1, 0, 6, 4, 2]
init_cost 3510572.0
best_cost 2292765.5
avg_cost 550934.8786213786
improvement 34.689688745879586
elapsed time 22.394913911819458


## C18V5

In [42]:

start_time = time.time()
c18v5_init_sol, c18v5_init_cost, feasibility, c, prob = run_problem('./Call_18_Vehicle_5.txt')
c18v5_init_cost = cost_function(c18v5_init_sol, prob)
c18v5_best_sol = c18v5_init_sol
c18v5_best_cost = c18v5_init_cost
c18v5_total_cost = c18v5_init_cost

# Force start with feasible solution
while(not feasibility):
    c18v5_init_sol, c18v5_init_cost, feasibility, c, prob = run_problem('./Call_18_Vehicle_5.txt')
    c18v5_init_cost = cost_function(c18v5_init_sol, prob)
    c18v5_best_sol = c18v5_init_sol
    c18v5_best_cost = c18v5_init_cost
    c18v5_total_cost = c18v5_init_cost

for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_18_Vehicle_5.txt')
    if feasibility:
        c18v5_total_cost += sol_cost
        if(sol_cost < c18v5_best_cost):
            c18v5_best_sol = solution
            c18v5_best_cost = sol_cost
end_time = time.time()

c18v5_avg_cost = c18v5_total_cost / (n+1)
print(c18v5_best_sol)
print('init_cost', c18v5_init_cost)
print('best_cost', c18v5_best_cost)
print('avg_cost', c18v5_avg_cost)
print('improvement', ((c18v5_init_cost - c18v5_best_cost) / c18v5_init_cost)*100)
print('elapsed time', end_time - start_time)

[0, 18, 18, 0, 14, 14, 0, 0, 0, 5, 9, 7, 13, 6, 11, 16, 8, 10, 4, 1, 17, 2, 15, 3, 12]
init_cost 4261270.0
best_cost 4261270.0
avg_cost 221589.5114885115
improvement 0.0
elapsed time 43.51043629646301


## C35V7 

In [7]:
start_time = time.time()
c35v7_init_sol, c35v7_init_cost, feasibility, c, prob = run_problem('./Call_35_Vehicle_7.txt')
c35v7_init_cost = cost_function(c35v7_init_sol, prob)
c35v7_best_sol = c35v7_init_sol
c35v7_best_cost = c35v7_init_cost
c35v7_total_cost = c35v7_init_cost

# Force start with feasible solution
while not feasibility:
    c35v7_init_sol, c35v7_init_cost, feasibility, c, prob = run_problem('./Call_35_Vehicle_7.txt')
    c35v7_init_cost = cost_function(c35v7_init_sol, prob)
    c35v7_best_sol = c35v7_init_sol
    c35v7_best_cost = c35v7_init_cost
    c35v7_total_cost = c35v7_init_cost
    
for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_35_Vehicle_7.txt')
    if feasibility:
        c35v7_total_cost += sol_cost
        if(sol_cost < c35v7_best_cost):
            c35v7_best_sol = solution
            c35v7_best_cost = sol_cost
end_time = time.time()

c35v7_avg_cost = c35v7_total_cost / (n+1)
print(c35v7_best_sol)
print('init_cost', c35v7_init_cost)
print('best_cost', c35v7_best_cost)
print('avg_cost', c35v7_avg_cost)
print('improvement', ((c35v7_init_cost - c35v7_best_cost) / c35v7_init_cost)*100)
print('elapsed time', end_time - start_time)

[0, 0, 0, 31, 0, 0, 0, 0, 15, 19, 26, 32, 8, 1, 33, 5, 9, 20, 2, 34, 4, 21, 28, 3, 16, 13, 31, 7, 29, 30, 24, 14, 12, 6, 35, 25, 18, 23, 27, 17, 11, 10, 22]
init_cost 9519498.0
best_cost 9238809.0
avg_cost 9538969.195304696
improvement 2.948569346829003
elapsed time 110.73691987991333


## C80V20

In [8]:
start_time = time.time()
c80v20_init_sol, c80v20_init_cost, feasibility, c, prob = run_problem('./Call_80_Vehicle_20.txt')
c80v20_init_cost = cost_function(c80v20_init_sol, prob)
c80v20_best_sol = c80v20_init_sol
c80v20_best_cost = c80v20_init_cost
c80v20_total_cost = c80v20_init_cost
while not f:
    c80v20_init_sol, c80v20_init_cost, feasibility, c, prob = run_problem('./Call_80_Vehicle_20.txt')
    c80v20_init_cost = cost_function(c80v20_init_sol, prob)
    c80v20_best_sol = c80v20_init_sol
    c80v20_best_cost = c80v20_init_cost
    c80v20_total_cost = c80v20_init_cost
    
for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_80_Vehicle_20.txt')
    if feasibility:
        c80v20_total_cost += sol_cost
        if(sol_cost < c80v20_best_cost):
            c80v20_best_sol = solution
            c80v20_best_cost = sol_cost
end_time = time.time()

c80v20_avg_cost = c80v20_total_cost / (n+1)
print(c80v20_best_sol)
print('init_cost', c80v20_init_cost)
print('best_cost', c80v20_best_cost)
print('avg_cost', c80v20_avg_cost)
print('improvement', ((c80v20_init_cost - c80v20_best_cost) / c80v20_init_cost)*100)
print('elapsed time', end_time - start_time)

[0, 0, 0, 0, 0, 0, 0, 39, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 15, 61, 76, 70, 7, 37, 36, 66, 12, 27, 64, 40, 41, 13, 47, 23, 32, 5, 19, 55, 56, 43, 38, 4, 78, 21, 63, 45, 9, 16, 22, 51, 3, 73, 79, 39, 57, 20, 75, 1, 31, 29, 25, 11, 26, 62, 72, 14, 60, 2, 59, 35, 67, 42, 8, 48, 34, 52, 10, 46, 33, 69, 30, 6, 58, 53, 28, 65, 68, 71, 17, 18, 80, 74, 77, 24, 49, 54, 50, 44]
init_cost 23564622.0
best_cost 23418242.0
avg_cost 23949009.984515484
improvement 0.6211854363715233
elapsed time 321.04793190956116


## C130V40

In [ ]:
start_time = time.time()
c130v40_init_sol, c130v40_init_cost, feasibility, c, prob = run_problem('./Call_130_Vehicle_40.txt')
c130v40_init_cost = cost_function(c130v40_init_sol, prob)
c130v40_best_sol = c130v40_init_sol
c130v40_best_cost = c130v40_init_cost
c130v40_total_cost = c130v40_init_cost
while not feasibility:
    c130v40_init_sol, c130v40_init_cost, feasibility, c, prob = run_problem('./Call_130_Vehicle_40.txt')
    c130v40_init_cost = cost_function(c130v40_init_sol, prob)
    c130v40_best_sol = c130v40_init_sol
    c130v40_best_cost = c130v40_init_cost
    c130v40_total_cost = c130v40_init_cost
    
for i in range(n):
    solution, sol_cost feasibility, c, prob = run_problem('./Call_130_Vehicle_40.txt')
    if feasibility:
        c130v40_total_cost += sol_cost
        if(sol_cost < c130v40_best_cost):
            c130v40_best_sol = solution
            c130v40_best_cost = sol_cost
end_time = time.time()

c130v40_avg_cost = c130v40_total_cost / (n+1)
print(c130v40_best_sol)
print('init_cost', c130v40_init_cost)
print('best_cost', c130v40_best_cost)
print('avg_cost', c130v40_avg_cost)
print('improvement', ((c130v40_init_cost - c130v40_best_cost) / c130v40_init_cost)*100)
print('elapsed time', end_time - start_time)

## C300V90

In [ ]:
start_time = time.time()
c300v90_init_sol, c300v90_init_cost, feasibility, c, prob = run_problem('./Call_300_Vehicle_90.txt')
c300v90_best_sol = c300v90_init_sol
c300v90_best_cost = c300v90_init_cost
c300v90_total_cost = c300v90_init_cost
while not feasibility:
    c300v90_init_sol, c300v90_init_cost, feasibility, c, prob = run_problem('./Call_300_Vehicle_90.txt')
    c300v90_best_sol = c300v90_init_sol
    c300v90_best_cost = c300v90_init_cost
    c300v90_total_cost = c300v90_init_cost
    
for i in range(n):
    solution, sol_cost, feasibility, c, prob = run_problem('./Call_300_Vehicle_90.txt')
    if feasibility:
        c300v90_total_cost += sol_cost
        if(sol_cost < c300v90_best_cost):
            c300v90_best_sol = solution
            c300v90_best_cost = sol_cost
end_time = time.time()

c300v90_avg_cost = c300v90_total_cost / (n+1)
print(c300v90_best_sol)
print('init_cost', c300v90_init_cost)
print('best_cost', c300v90_best_cost)
print('avg_cost', c300v90_avg_cost)
print('improvement', ((c300v90_init_cost - c300v90_best_cost) / c300v90_init_cost)*100)
print('elapsed time', end_time - start_time)